In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import roc_auc_score, confusion_matrix
import torchvision.transforms as transforms
import torchvision.datasets as datasets 
from torch.utils.data import DataLoader#designed for loading image data from a folder structure where each subfolder 
                                       #represents a different class or category
import time as time
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



In [2]:
import torchvision.ops as ops

In [3]:
import random
import numpy as np
# random.seed(10)

num_classes = 4
SEED = 1234  #its a value we take to generate a random value

random.seed(SEED)  ##random is a function which generates random value by taking initially an input
np.random.seed(SEED)  #by this we ensure that any random number generated while using numpy lib will be same for a given seed
                      #suppose we are taking different value for different epoch it can give different result
                      #to have a unbiased result we specify what random value we should use always
torch.manual_seed(SEED) ##it goes same for the torch to produce same random number
torch.cuda.manual_seed(SEED)  #it goes same for cuda as well
torch.backends.cudnn.deterministic = True # it increase gpu performance
print(random.random())

0.9664535356921388


In [4]:
import torch;
torch.__version__

'1.12.1+cu113'

In [5]:
import torchvision
torchvision.__version__

'0.13.1+cu113'

In [6]:
mobilenet = models.mobilenet_v3_large(pretrained=True) #giving the model which we want to train
model = mobilenet

model.classifier[-1] = nn.Linear(1280, num_classes)  #changing the outputclasses of last layer

print(model.classifier)



C:\Users\anark\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\anark\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:

model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [8]:
# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#check if gpu is available 

In [9]:
# Define the training parameters

num_epochs = 2
batch_size =2
learning_rate = 0.001

# Define the transformation for data preprocessing
transform_v1 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
## we use compose so that we can do many transformation


In [10]:
train_dataset = datasets.ImageFolder(r'D:\small_dataset\training',transform=transform_v1) #designed for loading imagedata
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) ##it is used when we want to train the
                                                                                      #data in minibatches and to shuffle

val_dataset = datasets.ImageFolder(r'D:\small_dataset\validation',transform=transform_v1)
val_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(r'D:\small_dataset\testing',transform=transform_v1)
test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
# Create an instance of the modified ResNet model
#model = ModifiedResNet(resnet, additional_layers).to(device)
model = mobilenet.to(device) #it is used to move the model in specific device
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss() #to calculate the loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  #its an optimizer


In [12]:
# Variables to keep track of the best model
best_accuracy = 0.0
best_model_weights = None

In [13]:
# Initialize lists to store metrics for each epoch
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
val_roc_aucs = []

In [14]:
# Training loop
for epoch in range(num_epochs):
    start_time = time.time() #here we start the time
    running_loss = 0.0   #we initialse the training loss
    correct_predictions = 0  #we initialise the predictions
    total_samples = 0  

    model.train()  # Set the model to training mode
# here our whole concept is we want to iterate through whole dataset
    for i, (images, labels) in enumerate(train_loader):   #enumerate here iterate through a batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_correct = 0
    val_total = 0
    val_predictions = []
    val_true_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            val_outputs = model(images)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (val_predicted == labels).sum().item()

            val_predictions.extend(val_outputs.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = val_correct / val_total
    val_loss = criterion(val_outputs, labels).item()
    
#     # Calculate ROC AUC for validation set
#     val_roc_auc = []
#     for class_index in range(num_classes):
#         fpr, tpr, _ = roc_curve(
#             (np.array(val_true_labels) == class_index).astype(int),
#             np.array(val_predictions)[:, class_index]
#         )
#         val_roc_auc.append(auc(fpr, tpr))
        
        
        
     # Calculate ROC AUC for validation set
    val_roc_aucs = []
    for class_index in range(num_classes):
        val_true_class = (np.array(val_true_labels) == class_index).astype(int)
        val_pred_class = np.array(val_predictions)[:, class_index]
        val_roc_aucs.append(roc_auc_score(val_true_class, val_pred_class))   

    # Store metrics
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    val_roc_aucs.append(val_roc_aucs)


    end_time = time.time()
    epoch_time = end_time - start_time
    
    # Convert epoch_time to hours, minutes, and seconds
    epoch_hours = int(epoch_time // 3600)
    epoch_minutes = int((epoch_time % 3600) // 60)
    epoch_seconds = int(epoch_time % 60)

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.4f},Validation Loss:{val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Time: {epoch_seconds:.2f} seconds")

    # Save the model if it's the best so far
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model_weights = model.state_dict()
        
        

# Save the best model
torch.save(best_model_weights, 'best_model.pth')


Epoch [1/2], Training Loss: 1.0752, Training Accuracy: 0.5429,Validation Loss:0.8052, Validation Accuracy: 0.4107, Time: 1.00 seconds
Epoch [2/2], Training Loss: 0.7710, Training Accuracy: 0.6911,Validation Loss:2.5945, Validation Accuracy: 0.4982, Time: 33.00 seconds


In [ ]:
# Plot training and validation loss curves
plt.figure(figsize=(8,8))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(8,8))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


# # Plot ROC AUC curve for validation set
# fpr, tpr, _ = roc_curve(val_true_labels, np.array(val_predictions).ravel())
# plt.figure(figsize=(8, 8))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % val_roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc='lower right')
# plt.show()

# val_conf_matrix = confusion_matrix(val_true_labels, val_predicted_labels)

# # 

# print("Validation Confusion Matrix:")
# print(val_conf_matrix)


In [ ]:
# torch.cuda.empty_cache()

In [15]:
#testing

def predict_dl(model,data):
    yno_pred = []
    yno_true = []
    for i, (images,labels) in enumerate(data):
        images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        #print(pred)
        pred = pred.data.cpu()
        yno_pred.extend(list(pred.numpy()))
        yno_true.extend(list(labels.numpy()))
    return np.array(yno_pred), np.array(yno_true)


In [16]:
yno_pred, yno_true = predict_dl(model,test_loader)
# model.eval()


In [17]:
pd.DataFrame(confusion_matrix(yno_true,yno_pred, labels=np.arange(0,4)))

,0,1,2,3
0,0,140,0,0
1,0,139,0,1
2,0,0,140,0
3,0,140,0,0
